In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
# Settings
methods = ["ele","da","jb","sig","esig"]
method = "ele"
eBeam = 5
pBeam = 41
beamConfig = f"{eBeam}x{pBeam}"
binning = 'StandardBinning' # 'StandardBinning' or 'HeraBinning' 
lumi = 10. # target luminosity in fb-1 (requires that EvalYields.ipynb be run with this target lumi)
pdfset = "NNPDF31_nnlo_as_0118_proton"

In [3]:
# SETTINGS FOR SYSTEMATICS
# YR inspired
point_to_point_err_low_percent = 1.2
point_to_point_err_high_percent = 4.2
normalisation_err_low_percent = 1
normalisation_err_high_percent = 1.5
# HERA inspired
# point_to_point_err_low_percent = 1.
# point_to_point_err_high_percent = 1.9 
# normalisation_err_low_percent = 0.
# normalisation_err_high_percent = 3.4
y_degradation_threshold = 0.01 # Threshold value below which systematics are degraded as 1 / y (0.1 or 0.05 for now) (0.01 to avoid)

In [4]:
binning_df = pd.read_csv(f"../Tables/{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_bins.csv")
binning_df

,Q2,x,y,gen,rec,gen_rec,purity,stability
0,1.258925,0.001995,0.769460,4.634915e+07,3.107802e+07,2.826037e+07,0.909336,0.609728
1,1.258925,0.003162,0.485497,7.481432e+07,6.668335e+07,5.475309e+07,0.821091,0.731853
2,1.258925,0.005012,0.306328,9.214961e+07,9.735445e+07,6.993744e+07,0.718379,0.758955
3,1.258925,0.007943,0.193280,1.118488e+08,1.168375e+08,7.879675e+07,0.674413,0.704494
4,1.258925,0.012589,0.121951,1.247757e+08,1.311817e+08,7.351524e+07,0.560408,0.589179
...,...,...,...,...,...,...,...,...
94,199.526231,0.501187,0.485497,1.841277e+05,1.344364e+05,1.271453e+05,0.945765,0.690528
95,199.526231,0.794328,0.306328,1.951199e+04,1.631456e+04,1.314721e+04,0.805857,0.673801
96,316.227766,0.501187,0.769460,9.393487e+04,6.303643e+04,6.094156e+04,0.966767,0.648764
97,316.227766,0.794328,0.485497,1.030728e+04,7.570084e+03,6.030675e+03,0.796646,0.585089


In [5]:
xsec_df = pd.read_csv(f"../Tables/{binning}_{method}_Ee{eBeam}_Ep{pBeam}_{pdfset}_XsecPredictions.csv")
xsec_df

,Q2,x,y,sigma_red_0,sigma_red_unc
0,1.258925,0.001995,0.769460,0.213504,0.012251
1,1.258925,0.003162,0.485497,0.206107,0.009242
2,1.258925,0.005012,0.306328,0.203583,0.008896
3,1.258925,0.007943,0.193280,0.203816,0.007778
4,1.258925,0.012589,0.121951,0.205017,0.007709
...,...,...,...,...,...
94,199.526231,0.501187,0.485497,0.084016,0.001317
95,199.526231,0.794328,0.306328,0.004552,0.000723
96,316.227766,0.501187,0.769460,0.080542,0.001267
97,316.227766,0.794328,0.485497,0.004164,0.000693


In [6]:
# Tolerances for dataframe merging
tol_x  = 1e-5
tol_Q2 = 1e-3
tol_y  = 1e-4

def merge_dataframes(binning_df, xsec_df,
                   tol_x=1e-6, tol_Q2=1e-6, tol_y=1e-6):
    """
    Merge binning and xsec dataframes on (Q2, x, y) values, within tolerances.
    Keep only rows with a matching xsec_df entry.
    """

    # sort xsec_df - speeds up search
    xsec_sorted = xsec_df.sort_values(["Q2", "x"]).reset_index(drop=True)
    # xsec_sorted = xsec_df

    matched_rows = []
    unmatched_count = 0

    for _, row in binning_df.iterrows():
        # find candidate matches within tolerance
        candidates = xsec_sorted[
            (abs(xsec_sorted["Q2"] - row["Q2"]) < tol_Q2) &
            (abs(xsec_sorted["x"]  - row["x"])  < tol_x)  &
            (abs(xsec_sorted["y"]  - row["y"])  < tol_y)
        ]

        if len(candidates) == 0:
            unmatched_count += 1
            continue   # drop row (no match)

        # If multiple matches exist, take the closest one
        cand = candidates.iloc[0]

        merged_row = {
            "Q2": row["Q2"],
            "x": row["x"],
            "y": row["y"],
            "gen": row["gen"],
            "rec": row["rec"],
            "gen_rec": row["gen_rec"],
            "purity": row["purity"],
            "stability": row["stability"],
            "sigma_red_0": cand["sigma_red_0"],
            "sigma_red_unc": cand["sigma_red_unc"],
        }

        matched_rows.append(merged_row)

    # Convert to dataframe
    merged_df = pd.DataFrame(matched_rows)

    # Print warning if rows were dropped
    if unmatched_count > 0:
        print(f"Warning: {unmatched_count} rows from binning_df had no matching xsec_df entry and were dropped.")

    return merged_df


merged_df = merge_dataframes(binning_df, xsec_df, tol_x, tol_Q2, tol_y)
merged_df

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red_0,sigma_red_unc
0,1.258925,0.001995,0.769460,4.634915e+07,3.107802e+07,2.826037e+07,0.909336,0.609728,0.213504,0.012251
1,1.258925,0.003162,0.485497,7.481432e+07,6.668335e+07,5.475309e+07,0.821091,0.731853,0.206107,0.009242
2,1.258925,0.005012,0.306328,9.214961e+07,9.735445e+07,6.993744e+07,0.718379,0.758955,0.203583,0.008896
3,1.258925,0.007943,0.193280,1.118488e+08,1.168375e+08,7.879675e+07,0.674413,0.704494,0.203816,0.007778
4,1.258925,0.012589,0.121951,1.247757e+08,1.311817e+08,7.351524e+07,0.560408,0.589179,0.205017,0.007709
...,...,...,...,...,...,...,...,...,...,...
94,199.526231,0.501187,0.485497,1.841277e+05,1.344364e+05,1.271453e+05,0.945765,0.690528,0.084016,0.001317
95,199.526231,0.794328,0.306328,1.951199e+04,1.631456e+04,1.314721e+04,0.805857,0.673801,0.004552,0.000723
96,316.227766,0.501187,0.769460,9.393487e+04,6.303643e+04,6.094156e+04,0.966767,0.648764,0.080542,0.001267
97,316.227766,0.794328,0.485497,1.030728e+04,7.570084e+03,6.030675e+03,0.796646,0.585089,0.004164,0.000693


In [7]:
# Statistical uncertainties (the only thing coming from ePIC currently)

def calc_sigma_stat(row):
    """
    Get statistical uncertainty IN PERCENT
    """
    return 100/np.sqrt(row['rec'])

merged_df['sigma_stat'] = merged_df.apply(calc_sigma_stat, axis=1)
merged_df

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red_0,sigma_red_unc,sigma_stat
0,1.258925,0.001995,0.769460,4.634915e+07,3.107802e+07,2.826037e+07,0.909336,0.609728,0.213504,0.012251,0.017938
1,1.258925,0.003162,0.485497,7.481432e+07,6.668335e+07,5.475309e+07,0.821091,0.731853,0.206107,0.009242,0.012246
2,1.258925,0.005012,0.306328,9.214961e+07,9.735445e+07,6.993744e+07,0.718379,0.758955,0.203583,0.008896,0.010135
3,1.258925,0.007943,0.193280,1.118488e+08,1.168375e+08,7.879675e+07,0.674413,0.704494,0.203816,0.007778,0.009251
4,1.258925,0.012589,0.121951,1.247757e+08,1.311817e+08,7.351524e+07,0.560408,0.589179,0.205017,0.007709,0.008731
...,...,...,...,...,...,...,...,...,...,...,...
94,199.526231,0.501187,0.485497,1.841277e+05,1.344364e+05,1.271453e+05,0.945765,0.690528,0.084016,0.001317,0.272735
95,199.526231,0.794328,0.306328,1.951199e+04,1.631456e+04,1.314721e+04,0.805857,0.673801,0.004552,0.000723,0.782911
96,316.227766,0.501187,0.769460,9.393487e+04,6.303643e+04,6.094156e+04,0.966767,0.648764,0.080542,0.001267,0.398294
97,316.227766,0.794328,0.485497,1.030728e+04,7.570084e+03,6.030675e+03,0.796646,0.585089,0.004164,0.000693,1.149343


In [8]:
sigma_p2p_opt_list = [point_to_point_err_low_percent]*len(merged_df.index)
sigma_p2p_pess_list = [point_to_point_err_high_percent]*len(merged_df.index)
sigma_norm_opt_list = [normalisation_err_low_percent]*len(merged_df.index)
sigma_norm_pess_list = [normalisation_err_high_percent]*len(merged_df.index)

merged_df["sigma_p2p_opt"] = sigma_p2p_opt_list
merged_df["sigma_p2p_pess"] = sigma_p2p_pess_list
merged_df["sigma_norm_opt"] = sigma_norm_opt_list
merged_df["sigma_norm_pess"] = sigma_norm_pess_list

merged_df = merged_df.rename(columns={"sigma_red_0": "sigma_red", "sigma_red_unc": "sigma_red_pdf_unc"}) # rename for convenience
#merged_df = merged_df.rename(columns={"sigma_red_unc": "sigma_red_pdf_unc"})

merged_df.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_all_columns_grid.csv",index=False)
merged_df

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p_opt,sigma_p2p_pess,sigma_norm_opt,sigma_norm_pess
0,1.258925,0.001995,0.769460,4.634915e+07,3.107802e+07,2.826037e+07,0.909336,0.609728,0.213504,0.012251,0.017938,1.2,4.2,1,1.5
1,1.258925,0.003162,0.485497,7.481432e+07,6.668335e+07,5.475309e+07,0.821091,0.731853,0.206107,0.009242,0.012246,1.2,4.2,1,1.5
2,1.258925,0.005012,0.306328,9.214961e+07,9.735445e+07,6.993744e+07,0.718379,0.758955,0.203583,0.008896,0.010135,1.2,4.2,1,1.5
3,1.258925,0.007943,0.193280,1.118488e+08,1.168375e+08,7.879675e+07,0.674413,0.704494,0.203816,0.007778,0.009251,1.2,4.2,1,1.5
4,1.258925,0.012589,0.121951,1.247757e+08,1.311817e+08,7.351524e+07,0.560408,0.589179,0.205017,0.007709,0.008731,1.2,4.2,1,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,199.526231,0.501187,0.485497,1.841277e+05,1.344364e+05,1.271453e+05,0.945765,0.690528,0.084016,0.001317,0.272735,1.2,4.2,1,1.5
95,199.526231,0.794328,0.306328,1.951199e+04,1.631456e+04,1.314721e+04,0.805857,0.673801,0.004552,0.000723,0.782911,1.2,4.2,1,1.5
96,316.227766,0.501187,0.769460,9.393487e+04,6.303643e+04,6.094156e+04,0.966767,0.648764,0.080542,0.001267,0.398294,1.2,4.2,1,1.5
97,316.227766,0.794328,0.485497,1.030728e+04,7.570084e+03,6.030675e+03,0.796646,0.585089,0.004164,0.000693,1.149343,1.2,4.2,1,1.5


In [9]:
# Write optimistic grid
opt_df = merged_df[['Q2','x','y','purity','stability','sigma_red','sigma_red_pdf_unc','sigma_stat','sigma_p2p_opt','sigma_norm_opt']]
opt_df = opt_df.rename(columns={"sigma_p2p_opt":"sigma_p2p","sigma_norm_opt":"sigma_norm"})
opt_df.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_optimistic_grid.csv",index=False)
opt_df

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,1.258925,0.001995,0.769460,0.909336,0.609728,0.213504,0.012251,0.017938,1.2,1
1,1.258925,0.003162,0.485497,0.821091,0.731853,0.206107,0.009242,0.012246,1.2,1
2,1.258925,0.005012,0.306328,0.718379,0.758955,0.203583,0.008896,0.010135,1.2,1
3,1.258925,0.007943,0.193280,0.674413,0.704494,0.203816,0.007778,0.009251,1.2,1
4,1.258925,0.012589,0.121951,0.560408,0.589179,0.205017,0.007709,0.008731,1.2,1
...,...,...,...,...,...,...,...,...,...,...
94,199.526231,0.501187,0.485497,0.945765,0.690528,0.084016,0.001317,0.272735,1.2,1
95,199.526231,0.794328,0.306328,0.805857,0.673801,0.004552,0.000723,0.782911,1.2,1
96,316.227766,0.501187,0.769460,0.966767,0.648764,0.080542,0.001267,0.398294,1.2,1
97,316.227766,0.794328,0.485497,0.796646,0.585089,0.004164,0.000693,1.149343,1.2,1


In [10]:
# Write pessimistic grid
pess_df = merged_df[['Q2','x','y','purity','stability','sigma_red','sigma_red_pdf_unc','sigma_stat','sigma_p2p_pess','sigma_norm_pess']]
pess_df = pess_df.rename(columns={"sigma_p2p_pess":"sigma_p2p","sigma_norm_pess":"sigma_norm"})
pess_df.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_pessimistic_grid.csv",index=False)
pess_df

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,1.258925,0.001995,0.769460,0.909336,0.609728,0.213504,0.012251,0.017938,4.2,1.5
1,1.258925,0.003162,0.485497,0.821091,0.731853,0.206107,0.009242,0.012246,4.2,1.5
2,1.258925,0.005012,0.306328,0.718379,0.758955,0.203583,0.008896,0.010135,4.2,1.5
3,1.258925,0.007943,0.193280,0.674413,0.704494,0.203816,0.007778,0.009251,4.2,1.5
4,1.258925,0.012589,0.121951,0.560408,0.589179,0.205017,0.007709,0.008731,4.2,1.5
...,...,...,...,...,...,...,...,...,...,...
94,199.526231,0.501187,0.485497,0.945765,0.690528,0.084016,0.001317,0.272735,4.2,1.5
95,199.526231,0.794328,0.306328,0.805857,0.673801,0.004552,0.000723,0.782911,4.2,1.5
96,316.227766,0.501187,0.769460,0.966767,0.648764,0.080542,0.001267,0.398294,4.2,1.5
97,316.227766,0.794328,0.485497,0.796646,0.585089,0.004164,0.000693,1.149343,4.2,1.5


In [20]:
# added this after the other degrade y function to keep rather than replace the existing sigma_p2p
def degrade_y_keep_column(row):
    """
    Degrade p2p systematics as 1/y below y_degradation_threshold
    """
    if row['y'] < y_degradation_threshold:
        row['sigma_p2p_pess'] *= (y_degradation_threshold/row['y'])
    return row

merged_df_y_degraded = merged_df.apply(degrade_y_keep_column, axis=1)

merged_df_y_degraded.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_all_columns_y_degraded_below_{y_degradation_threshold}_grid.csv",index=False)
merged_df_y_degraded

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p_opt,sigma_p2p_pess,sigma_norm_opt,sigma_norm_pess
0,2.0,0.0005,0.769231,5.807235e+06,2.495847e+06,2.277122e+06,0.912364,0.392118,0.430704,0.033821,0.063298,1.2,4.2,2.5,4.3
1,2.0,0.0008,0.480769,6.959221e+06,4.508726e+06,3.895783e+06,0.864054,0.559802,0.395696,0.021240,0.047095,1.2,4.2,2.5,4.3
2,2.0,0.0010,0.384615,7.505168e+06,6.376836e+06,4.852149e+06,0.760902,0.646508,0.384098,0.019121,0.039600,1.2,4.2,2.5,4.3
3,2.0,0.0020,0.192308,7.962894e+06,8.145345e+06,4.818906e+06,0.591615,0.605170,0.348735,0.011688,0.035038,1.2,4.2,2.5,4.3
4,2.0,0.0032,0.120192,7.838873e+06,8.965223e+06,3.631120e+06,0.405023,0.463220,0.330113,0.010957,0.033398,1.2,4.2,2.5,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,1200.0,0.6500,0.355030,3.376787e+02,2.179881e+02,1.514408e+02,0.694720,0.448476,0.020038,0.000768,6.773039,1.2,4.2,2.5,4.3
244,1500.0,0.4000,0.721154,1.127719e+03,3.861512e+02,3.316341e+02,0.858819,0.294075,0.140760,0.001950,5.088869,1.2,4.2,2.5,4.3
245,1500.0,0.6500,0.443787,2.677578e+02,1.364552e+02,9.605954e+01,0.703964,0.358755,0.020038,0.000892,8.560616,1.2,4.2,2.5,4.3
246,2000.0,0.6500,0.591716,2.280597e+02,9.925826e+01,7.955952e+01,0.801541,0.348854,0.020489,0.001199,10.037295,1.2,4.2,2.5,4.3


In [21]:
def degrade_y(row):
    """
    Degrade p2p systematics as 1/y below y_degradation_threshold
    """
    if row['y'] < y_degradation_threshold:
        row['sigma_p2p'] *= (y_degradation_threshold/row['y'])
    return row

# Write pessimistic y degraded grid
# if method == "ele":
pess_df_y_degraded = merged_df_y_degraded[['Q2','x','y','purity','stability','sigma_red','sigma_red_pdf_unc','sigma_stat','sigma_p2p_pess','sigma_norm_pess']]
pess_df_y_degraded = pess_df_y_degraded.rename(columns={"sigma_p2p_pess":"sigma_p2p","sigma_norm_pess":"sigma_norm"})   
# degrade p2p systematics as 1/y
    # pess_df_y_degraded = pess_df.apply(degrade_y, axis=1)

pess_df_y_degraded.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_pessimistic_y_degraded_below_{y_degradation_threshold}_grid.csv",index=False)
pess_df_y_degraded

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,2.0,0.0005,0.769231,0.912364,0.392118,0.430704,0.033821,0.063298,4.2,4.3
1,2.0,0.0008,0.480769,0.864054,0.559802,0.395696,0.021240,0.047095,4.2,4.3
2,2.0,0.0010,0.384615,0.760902,0.646508,0.384098,0.019121,0.039600,4.2,4.3
3,2.0,0.0020,0.192308,0.591615,0.605170,0.348735,0.011688,0.035038,4.2,4.3
4,2.0,0.0032,0.120192,0.405023,0.463220,0.330113,0.010957,0.033398,4.2,4.3
...,...,...,...,...,...,...,...,...,...,...
243,1200.0,0.6500,0.355030,0.694720,0.448476,0.020038,0.000768,6.773039,4.2,4.3
244,1500.0,0.4000,0.721154,0.858819,0.294075,0.140760,0.001950,5.088869,4.2,4.3
245,1500.0,0.6500,0.443787,0.703964,0.358755,0.020038,0.000892,8.560616,4.2,4.3
246,2000.0,0.6500,0.591716,0.801541,0.348854,0.020489,0.001199,10.037295,4.2,4.3


In [22]:
pess_df_y_degraded[:30]

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,2.0,0.0005,0.769231,0.912364,0.392118,0.430704,0.033821,0.063298,4.200000,4.3
1,2.0,0.0008,0.480769,0.864054,0.559802,0.395696,0.021240,0.047095,4.200000,4.3
2,2.0,0.0010,0.384615,0.760902,0.646508,0.384098,0.019121,0.039600,4.200000,4.3
3,2.0,0.0020,0.192308,0.591615,0.605170,0.348735,0.011688,0.035038,4.200000,4.3
4,2.0,0.0032,0.120192,0.405023,0.463220,0.330113,0.010957,0.033398,4.200000,4.3
5,2.0,0.0050,0.076923,0.259386,0.319583,0.319143,0.010861,0.032172,4.200000,4.3
6,2.0,0.0080,0.048077,0.168866,0.209627,0.312911,0.009377,0.032172,4.200000,4.3
7,2.0,0.0130,0.029586,0.152683,0.143387,0.309573,0.008761,0.037460,4.200000,4.3
8,2.0,0.0200,0.019231,0.143025,0.086905,0.309106,0.007784,0.046516,4.200000,4.3
9,2.0,0.0320,0.012019,0.146152,0.057086,0.310522,0.007306,0.057478,4.200000,4.3
